### 보일러 시공서류 OCR 테스트 - 한화손해보험
- 작성일 : 2025. 4/29 

In [1]:
import fitz  
import os
import pandas as pd
import json
import requests
import uuid
import time

In [ ]:
# 1. raw 폴더의 PDF 파일을, 페이지당 png파일로 변환하여 scan폴더에 저장 

def pdf_to_images(pdf_path, output_folder):
    pdf = fitz.open(pdf_path)
    for i in range(len(pdf)):
        page = pdf[i]
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
        pix.save(f"{output_folder}/page_{i+1:03d}.png")
    pdf.close()


if __name__ == "__main__":
    pdf_to_images("raw/한화.pdf", "scan")


In [ ]:
# 2. Naver OCR을 이용하여, 텍스트 추출 

# NCP OCR API 설정
secret_key = 'R3BETXl4cHRPcW5yd1BaZVRtbExja3NscVZHa3d1dWU='
api_url = 'https://aoomy83n4j.apigw.ntruss.com/custom/v1/41471/2ff10fce68b3d901dbd3948e3daf67d14427ed3adeeb616ba608f87b373b9455/infer'

# 스캔 이미지가 저장된 폴더 지정
scan_folder = 'scan/'

# 각 파일의 텍스트 데이터를 저장할 딕셔너리
extracted_data = {}

# 폴더 내 모든 이미지 처리
for img_file in os.listdir(scan_folder):
    if img_file.endswith('.png'):
        img_path = os.path.join(scan_folder, img_file)
        filename = os.path.splitext(img_file)[0]  # 확장자 제거하여 파일명 추출
        
        # OCR API 호출
        files = [('file', open(img_path, 'rb'))]
        request_json = {
            'images': [
                {
                    'format': 'png',
                    'name': img_file,
                    'templateIds': [37203]
                }
            ],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000))
        }
        
        payload = {'message': json.dumps(request_json).encode('UTF-8')}
        headers = {'X-OCR-SECRET': secret_key}

        response = requests.request("POST", api_url, headers=headers, data=payload, files=files)
        result = response.json()

        # 각 파일별 텍스트 저장
        #extracted_data[filename] = [field['inferText'] for field in result['images'][0]['fields']]

In [ ]:
result

In [ ]:
# 3. 추출된 텍스트를 하나의 엑셀 파일에 결합하여 저장 
import csv

# 파싱된 정보
data = {
    "Name": "청옥초등학교",
    "Address_1": "평택시 청북읍 안청로1길 76",
    "Address_2": "(옥길리 1103) 좌측 1호기",
    "Tel_no": "031-684-2704",
    "Maker_nm": "(주) 경 동나비엔",
    "Maker_no": "7404 4248200 8115",
    "Model_nm": "NPW- 48KS",
    "Heat_power": "50,000",
    "Set_date": "2024 년 10 월 26 일",
    "etc": "가스온수기라 co경보기 설치 의무 아님"
}

# CSV로 저장
csv_filename = "boiler_info.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=data.keys())
    writer.writeheader()
    writer.writerow(data)

print(f"CSV 파일로 저장 완료: {csv_filename}")
